In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import ngmix
import galsim
import functools

import proplot as plot

In [44]:
def gen_data(rng):
    nse_fac = rng.uniform(low=10, high=20)
    fwhm = rng.uniform(low=0.5, high=1.75)
    psf_fwhm = rng.uniform(low=0.8, high=1.2)
    image_size = 48
    cen = (image_size - 1)/2
    gs_wcs = galsim.ShearWCS(
        0.263, 
        galsim.Shear(
            g1=rng.uniform(low=-0.01, high=0.01), 
            g2=rng.uniform(low=-0.01, high=0.01),
        ),
    ).jacobian()
    psf_off = rng.uniform(low=-1.5, high=1.5, size=2)

    gal = galsim.Exponential(
        half_light_radius=fwhm,
    ).shear(
        g1=rng.uniform(low=-0.4, high=0.4), 
        g2=rng.uniform(low=-0.4, high=0.4),
    ).withFlux(
        400)
    psf = galsim.Gaussian(
        fwhm=psf_fwhm
    ).shear(
        g1=rng.uniform(low=-0.4, high=0.4), 
        g2=rng.uniform(low=-0.4, high=0.4),
    )
    im = galsim.Convolve([gal, psf]).drawImage(
        nx=image_size,
        ny=image_size,
        wcs=gs_wcs).array
    im_true = gal.drawImage(
        nx=image_size,
        ny=image_size,
        wcs=gs_wcs,
        method='no_pixel').array

    noise = np.sqrt(np.sum(im**2)) / nse_fac
    im += np.random.normal(scale=noise, size=im.shape)

    wgt = np.ones_like(im) / noise**2
    scale = np.sqrt(gs_wcs.pixelArea())

    psf_im = psf.drawImage(
        nx=53,
        ny=53,
        wcs=gs_wcs,
        offset=psf_off).array

    psf_obs = ngmix.Observation(
        image=psf_im,
        jacobian=ngmix.Jacobian(row=26+psf_off[1], col=26+psf_off[0], wcs=gs_wcs)
    )

    obs = ngmix.Observation(
        image=im,
        jacobian=ngmix.Jacobian(row=cen, col=cen, wcs=gs_wcs),
        weight=wgt,
        psf=psf_obs,
    )

    return obs, psf_obs, im_true, im

In [61]:
from ngmix.runners import Runner, PSFRunner
from ngmix.guessers import SimplePSFGuesser, TFluxAndPriorGuesser
from ngmix.fitting import Fitter
from ngmix.gaussmom import GaussMom
from ngmix.bootstrap import Bootstrapper
from ngmix.joint_prior import PriorSimpleSep
import ngmix.priors


def run_gaussfit(obs, rng):
    prior = PriorSimpleSep(
        ngmix.priors.CenPrior(
            0.0,
            0.0,
            0.263,
            0.263,
            rng=rng,
        ),
        ngmix.priors.GPriorBA(0.2, rng=rng),
        ngmix.priors.TwoSidedErf(-1.0, 0.1, 1.0e+06, 1.0e+05, rng=rng),
        [ngmix.priors.TwoSidedErf(-100.0, 1.0, 1.0e+09, 1.0e+08, rng=rng)],
    )

    gm = GaussMom(1.2).go(obs)
    if gm['flags'] == 0:
        flux_guess = gm['flux']
        Tguess = gm['T']
    else:
        gm = GaussMom(1.2).go(obs.psf)
        if gm['flags'] == 0:
            Tguess = 2 * gm['T']
        else:
            Tguess = 2
        flux_guess = np.sum(obs.image)

    guesser = TFluxAndPriorGuesser(
        rng=rng, T=Tguess, flux=flux_guess, prior=prior,
    )
    psf_guesser = SimplePSFGuesser(rng=rng, guess_from_moms=True)

    fitter = Fitter(
        model='gauss',
        fit_pars={
            'maxfev': 2000,
            'xtol': 5.0e-5,
            'ftol': 5.0e-5,
        },
        prior=prior
    )
    psf_fitter = Fitter(
        model='gauss',
        fit_pars={
            'maxfev': 2000,
            'ftol': 1.0e-5,
            'xtol': 1.0e-5,
        },
    )
    psf_runner = PSFRunner(
        fitter=psf_fitter,
        guesser=psf_guesser,
        ntry=2,
    )
    runner = Runner(
        fitter=fitter,
        guesser=guesser,
        ntry=2,
    )
    return Bootstrapper(runner, psf_runner=psf_runner, ignore_failed_psf=False).go(obs)


In [78]:
import tqdm

rng = np.random.RandomState()

fwhm = 2
g1k = []
g1g = []
g1f = []
snrk = []
snrg = []
snrf = []
for _ in tqdm.trange(1000):
    obs = gen_data(rng)[0]    
    res_km = ngmix.ksigmamom.KSigmaMom(fwhm=fwhm).go(obs)
    res_gm = ngmix.gaussmom.GaussMom(fwhm=fwhm).go(obs)
    res_f = run_gaussfit(obs, rng)
    if res_gm['flags'] == 0 and res_km['flags'] == 0 and res_f['flags'] == 0:
        g1g.append(res_gm['e'][0])
        snrg.append(res_gm['s2n'])
        
        g1k.append(res_km['e'][0])
        snrk.append(res_km['s2n'])
        
        e1, e2 = ngmix.shape.g1g2_to_e1e2(*res_f['g'])
        g1f.append(e1)
        snrf.append(res_f['s2n'])

print("kmom g1 std|gmom g1 std|fit g1 std: %s|%s|%s" % (np.std(g1k), np.std(g1g), np.std(g1f)))
print("kmom s/n|gmom s/n|fit s/n: %s|%s|%s" % (np.median(snrk), np.median(snrg), np.median(snrf)))

100%|██████████| 1000/1000 [01:34<00:00, 10.53it/s]

kmom g1 std|gmom g1 std|fit g1 std: 0.3190997601093649|0.16700891016849567|0.3365893371571785
kmom s/n|gmom s/n|fit s/n: 13.682305251062711|14.068788830751446|15.023601117204153
